In [ ]:
#Run this cell only if executing in Colab
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import os
import glob
from pathlib import Path
import pandas as pd
import time

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, models, Model, optimizers
from tensorflow.keras import regularizers
from keras import backend as K
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Getting the dataset from kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = <...path to kaggle.json file>
%cd <...path to working directory>

In [ ]:
!kaggle datasets download -d crowww/a-large-scale-fish-dataset

In [ ]:
%ls

## Data extraction

In [ ]:
if not os.path.exists('Fish_Dataset'):
  %shell unzip a-large-scale-fish-dataset.zip -d ./
else:
  print('The dataset is already available')

In [ ]:
image_dir=Path('./Fish_Dataset/Fish_Dataset')
labels=list(image_dir.glob('**/*GT*/*.png'))

In [ ]:
images = list()

for folder in glob.glob('./Fish_Dataset/Fish_Dataset/*/**'):
  if not folder.endswith('GT'):
    path = folder+'/**'
    images.append(glob.glob(path))

images = [item for sublist in images for item in sublist]

In [ ]:
images = pd.Series(images, name='image').astype(str)
labels = pd.Series(labels,name='label').astype(str)
df = pd.concat((images, labels), axis=1)
df['fish_type'] = df['label'].apply(lambda x: x.split('/')[-3])

In [ ]:
df['fish_type'].value_counts()
# There 1000 images and 1000 labels for each class

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(images[0])
imgplot = plt.imshow(img)
plt.show()

## Model

In [ ]:
#TODO: Reshape the images
IMG_SHAPE = [128, 128, 3]

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False)

#Use the activations of these layers
layer_names = [
    'block_1_expand_relu',  #64x64
    'block_3_expand_relu',  #32x32
    'block_6_expand_relu',  #16x16
    'block_13_expand_relu', #8x8
    'block_16_project',     #4x4
]

base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

#Create a feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)
down_stack.trainable = False

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3), #4x4 -> 8x8
    pix2pix.upsample(256, 3), #8x8 -> 16x16
    pix2pix.upsample(128, 3), #16x16 -> 32x32
    pix2pix.upsample(64, 3),  #32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=IMG_SHAPE)

  #Downsampling through the model
  skips = down_stack(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])

  #Upsampling ans establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  #This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels,
      kernel_size=3,
      strides=2,
      padding='same'
  ) #64x64 -> 128x128

  x = last(x)
  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3

model = unet_model(output_channels=OUTPUT_CHANNELS)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)